In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [11]:
def property_detail_city(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:88.0) Gecko/20100101 Firefox/88.0'}
    response = requests.get(url,headers= headers)
    print(response.status_code)
    soup = BeautifulSoup(response.content, 'html.parser')
    results = list()
    city = soup.find('div' ,{'class':'css-7so23d e3vrfmg6','data-auto':"breadcrumb-text"}).text
    tags_lst = soup.find_all('div',{'class':"css-zuyt5t e1sx3tzs1",'data-auto': "listed-bulletin",'data-auto-bulletin-type':"bulletin"})
    for tag in tags_lst:
        property_dict = {}
        property_class = tag.find('div', {'data-auto': 'property-class'})
        property_dict['City'] = city
        property_dict['type'] = property_class.get_text() if property_class is not None else None
        property_dict['room_number'] = tag.find('div', {'data-auto': 'property-rooms'}).get_text() if tag.find('div', {'data-auto': 'property-rooms'}) is not None else None
        property_dict['Area'] = tag.find('div', {'data-auto': 'property-size'}).get_text() if tag.find('div', {'data-auto': 'property-size'}) is not None else None
        street_details = tag.find('div', {'data-auto': "property-address"}).get_text() if tag.find('div', {'data-auto': "property-address"}) is not None else None
        if street_details == None:
            property_dict['Street'] = None
            property_dict['number_in_street'] = None

        else:
            street = re.search(r'^([^,\d]+)', street_details)
            
            if street == None:
                property_dict['Street'] = None
            else:
                property_dict['Street'] = street.group()
            property_dict['number_in_street'] = int(re.findall(r'\d+', street_details)[0]) if re.findall(r'\d+', street_details) else None
            try:
                property_dict['city_area'] = ((re.split(r',\s*', street_details, maxsplit=1)[1])) if re.split(r',\s*', street_details, maxsplit=1)[1] else None
            except IndexError:
                property_dict['city_area'] = None
        price = tag.find('div', {'data-auto': 'property-price'}).get_text() if tag.find('div', {'data-auto': 'property-price'}) is not None else None
        if price == None:
            property_dict['price'] = None
        else:
            property_dict['price'] = re.search(r'(?<=₪)[\d,]+', price).group()

        property_url = "https://www.madlan.co.il" + tag.find('a').get('href')
        response1 =requests.get(property_url,headers= headers)
        soup1 = BeautifulSoup(response1.content,'html.parser')
        photos = soup1.find_all('div',{'class': 'css-18nlt6 e1r3dysu3'})
        property_dict['num_of_images'] = len(photos)
        property_dict['floor_out_of'] = soup1.find('div', {'data-auto': 'floor'}).get_text() if soup1.find('div', {'data-auto': 'floor'}) is not None else None
        blocks_lst = soup1.find_all('div',{'class':'css-1wpv10e e125ttrt3'})
        title_names = ['hasElevator','hasAirCondition','hasParking','hasBalcony','hasBars','hasMamad','hasStorage','handicapFriendly']
        for i in range (len(blocks_lst)-1):
            property_dict[title_names[i]] = blocks_lst[i].get_text()
            property_dict['condition'] = soup1.find('div', {'data-auto': 'unit-general-condition-value'}).get_text()
            property_dict['entranceDate'] = soup1.find('div', {'data-auto': 'unit-availability-value'}).get_text()
            property_dict['furniture'] = soup1.find('div', {'data-auto': 'unit-furniture-details-value'}).get_text()
            property_dict['description'] = soup1.find('div', {'class': 'css-v1qjdi ebqee3y2'}).get_text()


        results.append(property_dict)
    return results


In [13]:
url = "https://www.madlan.co.il/for-sale/%D7%A4%D7%AA%D7%97-%D7%AA%D7%A7%D7%95%D7%95%D7%94-%D7%99%D7%A9%D7%A8%D7%90%D7%9C?bbox=34.83621%2C32.06701%2C34.93027%2C32.11539&tracking_search_source=map"
df = pd.DataFrame(property_detail_city(url))
df

200


,City,type,room_number,Area,Street,number_in_street,city_area,price,num_of_images,floor_out_of,...,entranceDate,furniture,description,hasAirCondition,hasParking,hasBalcony,hasBars,hasMamad,hasStorage,handicapFriendly
0,פתח תקווה,דירת גן,5 חד׳,"120 מ""ר",עין גנים,1.0,עין גנים,"4,900,000",0,קומת קרקע,...,לא צויין,"למכירה, דירת גן, 122 בנוי פלוס 122 חצר.תקרה גב...","למכירה, דירת גן, 122 בנוי פלוס 122 חצר. תקרה ג...",יש מיזוג אויר,אין חניה,אין מרפסת,אין סורגים,יש ממ״ד,יש מחסן,נגיש לנכים
1,פתח תקווה,דירה,2 חד׳,"36 מ""ר",סולד הנריטה,5.0,אחדות,"1,600,000",0,קומה 2 מתוך 4,...,לא צויין,ראו בתמונות,למכירה ברח' הנרייטה סואלד בפתח תקווה דירת 2 חד...,אין מיזוג אויר,אין חניה,אין מרפסת,אין סורגים,אין ממ״ד,אין מחסן,לא נגיש לנכים
2,פתח תקווה,דירה,5 חד׳,"124 מ""ר",סנה משה,NaN,אם המושבות החדשה,"3,420,000",6,קומה 9 מתוך 21,...,מיידי,לא צויין,"בשכונת אם המושבות, דירת 5 חד', כ-124 מ""ר + כ-1...",יש מיזוג אויר,יש חנייה,יש מרפסת,אין סורגים,יש ממ״ד,יש מחסן,נגיש לנכים
3,פתח תקווה,דירה,4 חד׳,"86 מ""ר",טרומפלדור,NaN,כפר גנים א,"2,140,000",5,קומה 2 מתוך 7,...,לא צויין,מעוניינים לקנות נכס אבל עדיין לא מכרתם את שלכם...,למכירה! הדירה הכי אטרקטיבית בשכונה!בכפר גנים א...,יש מיזוג אויר,יש חנייה,יש מרפסת,אין סורגים,יש ממ״ד,יש מחסן,נגיש לנכים
4,פתח תקווה,דירה,4 חד׳,"94 מ""ר",מנחת שלמה,9.0,גני הדר,"2,580,000",12,קומה 2 מתוך 4,...,גמיש,חלקי,"דירה יפיפיה ומשופצת, אינסטלציה חדשה, מטבח ענק ...",יש מיזוג אויר,אין חניה,יש מרפסת,יש סורגים,יש ממ״ד,אין מחסן,נגיש לנכים
5,פתח תקווה,דירה,4.5 חד׳,"127 מ""ר",דנקנר מאיר,3.0,נווה גן,"3,620,000",5,קומה 1 מתוך 28,...,גמיש,ללא,***למכירה בפתח תקוה***\n*אחת הדירות היפות והמו...,יש מיזוג אויר,יש חנייה,יש מרפסת,אין סורגים,יש ממ״ד,יש מחסן,נגיש לנכים
6,פתח תקווה,דירה,3 חד׳,"55 מ""ר",טבריה,NaN,עמישב,"1,785,000",5,קומת קרקע,...,גמיש,לא צויין,למשקיעים ולזוגות צעירים זו ההזדמנות שלכם לשכפל...,יש מיזוג אויר,אין חניה,אין מרפסת,יש סורגים,אין ממ״ד,אין מחסן,לא נגיש לנכים
7,פתח תקווה,דירה,6 חד׳,"152 מ""ר",ראשון לציון,NaN,אם המושבות החדשה,"4,290,000",6,קומה 3 מתוך 18,...,גמיש,לא צויין,"למכירה ברחוב ראשון לציון באם במושבות פ""ת, \nדי...",יש מיזוג אויר,יש חנייה,יש מרפסת,יש סורגים,יש ממ״ד,יש מחסן,נגיש לנכים
8,פתח תקווה,פנטהאוז,6 חד׳,"160 מ""ר",גיסין אבשלום,23.0,נווה גן,"4,800,000",2,קומה 14 מתוך 14,...,מיידי,לא צויין,מציאה!!! למהירי החלטה פנטאוז פצצה! עם נוף פת...,יש מיזוג אויר,יש חנייה,יש מרפסת,אין סורגים,יש ממ״ד,יש מחסן,נגיש לנכים
9,פתח תקווה,דירה,4 חד׳,"110 מ""ר",סנה משה,35.0,אם המושבות החדשה,"3,160,000",0,קומה 6 מתוך 9,...,לא צויין,לא צויין,ללא תיווך! דירת 4 חדרים במיקום מרכזי באם המושב...,יש מיזוג אויר,אין חניה,אין מרפסת,אין סורגים,יש ממ״ד,יש מחסן,לא נגיש לנכים


In [14]:
df.to_csv('property.csv',encoding='utf-8-sig',index = False)